# Replication of Chen, Roll & Ross (1986)

Macro variables used:

1) Industrial Production
- IP is the rate of industrial production
- MP is the monthly growth rate (log(IP_t / IP_t-1))
- YP is the yearly growth rate
- Since MP is partial lagged, it should be lead by 1 month to make it contemporaneous
- YP is lead by 1 year (12 months)

2) Inflation
- Unanticipated inflation is defined as: UI_t = I_t - E[I_t |t-1]
- Where I_t is the first difference of the log of the CPI index
- The expectation is obtained from Fama and Gibbons (1984)
- Specifically, if RHO_t is the ex-post real interest rate, and TB_t-1 is last months (1 month) Treasury bill rate, then: TB_t-1 = E[RHO_t | t-1] + E[I_t | t-1]
- TB_t-1 - I_t = E[RHO_t | t-1] is the ex-post real return on the bill; Fama and Gibbons construct E[RHO_t | t-1] using an MA on the first difference *
- Then expected inflation is defined as: E[I_t |t-1] = TB_t-1 - E[RHO_t | t-1]
- The change in expected inflation is defined as: DEI_t = E[I_t+1 |t] - E[I_t |t-1]

3) Risk Premia
- UPR_t = BAA_t - LGB_t
- Where BAA is the return of a portfolio of sub-BAA bonds, and LBG is the return on a portfolio of long-term government bonds
- This is essentially the high-yield corporate bond spread 

4) Term Structure
- UTS_t = LGB_t - TB_t-1
- Unanticipated returns on long-bonds, but basically the slope of the yield curve

5) Market Indices
- EWNY is the return on the equally-weighted NYSE index
- VWNY is the return on the equally-weighted NYSE index

6) Consumption
- ...

7) Oil Prices
- OG is the first difference of the log of "Crude Petroleum" PPI


The total sample spans 371 months from January 1953 to November 1983.

The return of an individual stock is assumed to follow: 

$
R_{it} = a + b_{MP}MP_t + b_{DEI}DEI_t + b_{UI}UI_t + b_{UPR}UPR_t + b_{UTS}UTS_t + e_t
$

The estimation procedure follows:
1) Chose a sample of assets **
2) Regress the time-series of returns on the macro variables over the past 5 years, and obtain the betas
3) Regress the stock returns cross-sectionally on the estimated betas, running 12 regressions, one for each of the next 12 months
4) Repeat the prior steps to obtain time-series estimates of the risk-premia associated with each macro variable

** The set of assets were 20 equally-weighted portfolios formed on size


** Fama and Gibbons estimation

In [159]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import pandas_datareader.data as pdr
from pandas.tseries.offsets import *
plt.style.use('seaborn-whitegrid')

### Data

#### Predictor Data
Data predominately coming from FRED so it likely includes revisions

In [185]:
# Industrial production, CPI, BAA yield, Crude PPI, WTI spot price
fred_data = pdr.DataReader(['INDPRO','CPIAUCSL','BAA','WPU0561','WTISPLC'],'fred', start='1947-01-01')
fred_data.columns = ['IP',"CPI","BAA","OG","WTI"]
fred_data.index = fred_data.index + MonthEnd(0)
fred_data = fred_data.loc['1953-01-31':"1983-11-30"]

In [197]:
# Equity indices + Bond Returns from CRSP "Monthly Stock File Indexes"
equity = pd.read_csv("Equities.csv", index_col='MthCalDt', parse_dates=True)
equity = equity.pivot(columns="IndNm", values='COL1')
equity.columns = ['LGB', 'TB_1', 'TB_3', "EWNY", "VWNY"]
equity.index.name = 'Date'
equity.index = equity.index + MonthEnd(0)
equity = equity.loc['1953-01-31':"1983-11-30"]

In [199]:
# Merging both datasets
data = pd.merge(equity, fred_data, left_index=True, right_index=True)

#### Transformations

In [202]:
data

,LGB,TB_1,TB_3,EWNY,VWNY,IP,CPI,BAA,OG,WTI
Date,,,,,,,,,,
1953-01-31,0.001177,0.001620,0.001970,0.024706,-0.001644,19.2127,26.64,3.51,11.4,2.57
1953-02-28,-0.008685,0.001402,0.001351,0.012031,-0.001820,19.3205,26.59,3.53,11.7,2.57
1953-03-31,-0.008818,0.001892,0.002046,-0.010486,-0.012862,19.4822,26.63,3.57,11.7,2.57
1953-04-30,-0.010516,0.001636,0.001441,-0.024027,-0.027705,19.5630,26.69,3.65,11.7,2.57
1953-05-31,-0.014752,0.001722,0.001915,0.005896,0.006921,19.6708,26.70,3.78,11.7,2.57
...,...,...,...,...,...,...,...,...,...,...
1983-07-31,-0.047866,0.006917,0.007076,-0.020158,-0.030463,49.8873,99.80,13.39,92.1,31.66
1983-08-31,0.002665,0.008342,0.008917,-0.008660,0.012387,50.4843,100.10,13.64,92.1,31.91
1983-09-30,0.050744,0.007607,0.008945,0.024209,0.017475,51.2160,100.40,13.55,92.1,31.11


#### Stock Data

In [77]:
# # CRSP stock data including delisting returns
# crsp = pd.read_pickle('raw_crsp_data.pkl')
# crsp = crsp[crsp["date"] <= '1983-11-30']

In [76]:
# # Market Cap
# crsp['me']=  crsp['prc'].abs() * crsp['shrout'] 

# # Fills null returns with 0
# crsp['dlret'] = crsp['dlret'].fillna(0)
# crsp['ret'] = crsp['ret'].fillna(0)

# # Adjust returns for delisting
# crsp['retadj'] = (1+crsp['ret']) * (1+crsp['dlret']) - 1


# # Keeping share code 10 or 11, exchange code 1
# crsp = crsp[((crsp['shrcd']==10) | (crsp['shrcd']==11)) & (crsp['exchcd']==1)]

# # Dropping unnecessary variables
# crsp['date'] = crsp['jdate']
# crsp = crsp.drop(['dlret','dlstdt','prc','shrout','date','shrcd','exchcd'], axis=1)
# crsp = crsp.rename(columns={'jdate':'date'})
# crsp = crsp.sort_values(by=['date','permco','me'])

In [78]:
# crsp.to_pickle('crsp_data.pkl')
crsp = pd.read_pickle("crsp_data.pkl")

In [79]:
crsp

,permno,permco,ret,retx,date,me,retadj
610016,17670,74,0.016667,0.006410,1959-01-31,17034.500,0.016667
609920,18702,267,0.149254,0.149254,1959-01-31,28297.500,0.149254
749717,20714,584,0.129412,0.129412,1959-01-31,40512.000,0.129412
160741,11287,921,0.097403,0.097403,1959-01-31,37180.000,0.097403
766630,21151,994,0.009009,0.009009,1959-01-31,14826.000,0.009009
...,...,...,...,...,...,...,...
2207705,65699,25751,0.145570,0.145570,1983-11-30,479423.750,0.145570
1979082,58480,54172,0.072727,0.072727,1983-11-30,586047.000,0.072727
597424,17277,56246,0.154167,0.154167,1983-11-30,972997.125,0.154167
597467,17278,56247,0.076596,0.076596,1983-11-30,641861.000,0.076596


##### Test Asset Formation - 20 Size Portfolios
Subset stocks based on size, compute equally weighted portfolio return

In [87]:
size_breakpoints = pdr.DataReader('ME_Breakpoints','famafrench', start='1959-01-01', end='1983-11-30')[0]
size_breakpoints.columns = size_breakpoints.columns.map(str)

size_breakpoints

,Count,"(0, 5)","(5, 10)","(10, 15)","(15, 20)","(20, 25)","(25, 30)","(30, 35)","(35, 40)","(40, 45)",...,"(50, 55)","(55, 60)","(60, 65)","(65, 70)","(70, 75)","(75, 80)","(80, 85)","(85, 90)","(90, 95)","(95, 100)"
Date,,,,,,,,,,,,,,,,,,,,,
1959-01,1038,6.74,10.38,14.83,17.87,22.82,28.27,33.65,40.84,50.13,...,74.42,93.56,118.40,136.47,186.79,246.59,343.64,526.77,807.44,16472.26
1959-02,1035,6.93,11.10,14.94,18.74,23.59,28.84,34.96,42.36,51.79,...,75.70,96.73,120.09,140.28,194.85,258.93,349.34,534.88,820.46,17177.71
1959-03,1034,6.98,10.80,15.05,19.40,24.52,29.34,35.59,42.86,51.95,...,77.12,95.83,118.83,142.62,190.21,259.03,353.13,554.40,822.20,17018.98
1959-04,1037,7.40,11.16,15.55,19.38,24.77,30.53,36.54,44.28,53.66,...,79.21,96.91,121.66,148.90,197.32,257.52,360.00,550.25,864.66,17794.98
1959-05,1041,7.13,10.93,15.89,19.67,24.22,29.64,37.27,44.10,54.00,...,79.50,96.40,121.83,147.75,195.80,264.51,354.21,559.50,894.21,17636.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1983-07,1375,38.90,63.22,84.09,105.97,140.76,165.42,201.86,247.46,300.72,...,443.88,536.06,654.75,787.04,974.27,1206.51,1552.51,2207.25,3670.94,72889.47
1983-08,1370,37.63,60.86,80.83,103.69,132.94,157.94,197.53,238.63,294.66,...,434.69,528.65,655.83,781.76,989.85,1251.13,1574.47,2206.39,3733.79,72359.64
1983-09,1372,36.51,61.76,82.38,104.60,135.13,164.02,205.00,244.56,304.34,...,451.07,557.06,674.31,816.77,998.42,1253.42,1586.34,2287.09,3826.44,77082.91


In [99]:
crsp['me'] = crsp['me'] / 1000

### Estimation